In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [ ]:
# Set your sweep directory here
SWEEP_DIR = '/scratch/gpfs/TSILVER/de7281/interp_planning/sweeps/sweep_2025-11-23_22-44-48_job2533853'

sweep_path = Path(SWEEP_DIR)
print(f"Analyzing: {SWEEP_DIR}")
print(f"Exists: {sweep_path.exists()}")

In [ ]:
# Find all run directories
runs = []
for run_dir in sweep_path.iterdir():
    if run_dir.is_dir() and (run_dir / '.hydra').exists():
        runs.append(run_dir)

print(f"Found {len(runs)} runs")
print("\nFirst few runs:")
for run in runs[:5]:
    print(f"  {run.name}")

In [ ]:
# Extract results from all runs
results = []

for run_dir in runs:
    config_file = run_dir / '.hydra' / 'config.yaml'
    if not config_file.exists():
        continue
    
    config = OmegaConf.load(config_file)
    
    result = {
        'run_dir': str(run_dir.name),
        'k': config.model.k,
        'waypoint_type': config.planner.waypoint_type,
        'learning_rate': config.training.learning_rate,
        'l2_reg': config.training.l2_reg,
        'use_infonce': config.training.use_infonce,
    }
    
    # Parse log files
    for log_file in run_dir.glob('*.log'):
        try:
            with open(log_file) as f:
                for line in f:
                    if 'Success Rate:' in line:
                        result['success_rate'] = float(line.split(':')[1].strip().rstrip('%')) / 100
                    elif 'Efficiency:' in line:
                        result['efficiency'] = float(line.split(':')[1].strip().rstrip('%')) / 100
                    elif 'Mean Path Length:' in line:
                        result['mean_path_length'] = float(line.split(':')[1].strip())
        except: pass
    
    if 'success_rate' in result:
        results.append(result)

print(f"Parsed {len(results)} runs with results")

In [ ]:
df = pd.DataFrame(results)
df.head()

In [ ]:
# Top 10 by success rate
df.nlargest(10, 'success_rate')[['k', 'waypoint_type', 'learning_rate', 'success_rate', 'efficiency']]

In [ ]:
# Group by hyperparameters
df.groupby('waypoint_type')[['success_rate', 'efficiency']].agg(['mean', 'std'])

In [ ]:
# Save to CSV
df.to_csv(sweep_path / 'results.csv', index=False)
print(f"Saved to {sweep_path / 'results.csv'}")